PROJECT 2

Binomial Trees

In [1]:
import numpy as np
import pandas as pd
from math import exp, sqrt, pow, log
from scipy.stats import norm

In [2]:
def calculate (x, u, d, N, S_0) :
    index = x.name
    result = [0.0 if i>index else S_0*pow(u, index-i)*pow(d, i) for i in range(N)]
    return result

# Function that calculates Spot Price

In [3]:
def binomial_tree(S_0, N, u, d, delta_t) :
    arr = np.zeros((N,N))
    data = pd.DataFrame(arr)
    data = data.apply(lambda x:calculate(x, u, d, N, S_0), result_type='expand')
    return data

# Function that makes Binomial Tree
# Matrix Form; if column t index, then all possible cases of Spot Price at time t 

In [4]:
def compare_max(Euro_tree, i, N, K, call_put) :
    if call_put == 'call' :
        index = Euro_tree[N-i] - K <= 0.0
        Euro_tree[N-i][index] = 0.0
        Euro_tree[N-i][~index] = Euro_tree[N-i] - K
    elif call_put == 'put' :
        index = K - Euro_tree[N-i] <=0.0
        Euro_tree[N-i][index] = 0.0
        Euro_tree[N-i][~index] = K - Euro_tree[N-i]
        
# Function that finds Option Price at time i

In [5]:
def option_price(S_0, K, sig, N, q, tree, R, call_put, Euro_Amer) :
    Euro_tree = tree.copy()
    
    compare_max(Euro_tree, 1, N, K, call_put)
    for i in range (2, N+1):
        result = []
        for j in range(N) :
            if (N-i)<j :
                zero = np.zeros(i-1)
                result.extend(zero)
                break
            result.append((1/R)*(Euro_tree[N-i+1][j]*q+(1-q)*Euro_tree[N-i+1][j+1]))
        
        if Euro_Amer == "American" :
            compare_max(Euro_tree, i, N, K, call_put)
            Euro_tree[N-i][N-i+1:N] = 0
            temp = np.array(Euro_tree[N-i]) > np.array(result)
            idx = np.where(temp == True)
            for k in idx[0] :
                result[k] = Euro_tree[N-i][k]
                
        Euro_tree[N-i] = result
    return Euro_tree

# 'Euro_Amer' means European or American
# 'call_put' means Call Option or Put Option

In [6]:
S_0 = 70
K = 60
T = 10
r = 0.05
sig = 0.2
N = 10

delta_t = T/N
u = exp(sig*sqrt(delta_t))
d = 1/u
R = exp(r*delta_t)
q = (R - d) / (u - d)
BT = binomial_tree(S_0, N+1, u, d, delta_t)

In [7]:
BT

,0,1,2,3,4,5,6,7,8,9,10
0,70.0,85.498193,104.427729,127.548316,155.787865,190.279728,232.408185,283.863998,346.712270,423.475323,517.233927
1,0.0,57.311153,70.000000,85.498193,104.427729,127.548316,155.787865,190.279728,232.408185,283.863998,346.712270
2,0.0,0.000000,46.922403,57.311153,70.000000,85.498193,104.427729,127.548316,155.787865,190.279728,232.408185
3,0.0,0.000000,0.000000,38.416815,46.922403,57.311153,70.000000,85.498193,104.427729,127.548316,155.787865
4,0.0,0.000000,0.000000,0.000000,31.453027,38.416815,46.922403,57.311153,70.000000,85.498193,104.427729
5,0.0,0.000000,0.000000,0.000000,0.000000,25.751561,31.453027,38.416815,46.922403,57.311153,70.000000
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,21.083595,25.751561,31.453027,38.416815,46.922403
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.261787,21.083595,25.751561,31.453027
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.132756,17.261787,21.083595
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.570922,14.132756


In [8]:
Euro_call = option_price(S_0, K, sig, N+1, q, BT, R, 'call', 'European')

C:\Users\김수빈\AppData\Local\Temp\ipykernel_32960\561204226.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Euro_tree[N-i][index] = 0.0
C:\Users\김수빈\AppData\Local\Temp\ipykernel_32960\561204226.py:5: FutureWarning: ChainedAssignmentError: 

In [9]:
Euro_call

,0,1,2,3,4,5,6,7,8,9,10
0,36.058008,48.676653,64.91315,85.527300,111.393883,143.551681,183.284339,232.221519,292.422025,366.401557,457.233927
1,0.000000,23.186149,32.39109,44.614392,60.550904,80.957396,106.664020,138.637249,178.117940,226.790232,286.712270
2,0.000000,0.000000,13.41823,19.614618,28.245900,40.006623,55.645080,75.905837,101.497620,133.205963,172.408185
3,0.000000,0.000000,0.00000,6.577145,10.197388,15.598675,23.486370,34.704670,50.137484,70.474551,95.787865
4,0.000000,0.000000,0.00000,0.000000,2.427039,4.052221,6.710464,11.002974,17.822105,28.424428,44.427729
5,0.000000,0.000000,0.00000,0.000000,0.000000,0.500220,0.910602,1.657664,3.017618,5.493285,10.000000
6,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
Amer_call = option_price(S_0, K, sig, N+1, q, BT, R, 'call', 'European')

C:\Users\김수빈\AppData\Local\Temp\ipykernel_32960\561204226.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Euro_tree[N-i][index] = 0.0
C:\Users\김수빈\AppData\Local\Temp\ipykernel_32960\561204226.py:5: FutureWarning: ChainedAssignmentError: 

In [11]:
Amer_call

,0,1,2,3,4,5,6,7,8,9,10
0,36.058008,48.676653,64.91315,85.527300,111.393883,143.551681,183.284339,232.221519,292.422025,366.401557,457.233927
1,0.000000,23.186149,32.39109,44.614392,60.550904,80.957396,106.664020,138.637249,178.117940,226.790232,286.712270
2,0.000000,0.000000,13.41823,19.614618,28.245900,40.006623,55.645080,75.905837,101.497620,133.205963,172.408185
3,0.000000,0.000000,0.00000,6.577145,10.197388,15.598675,23.486370,34.704670,50.137484,70.474551,95.787865
4,0.000000,0.000000,0.00000,0.000000,2.427039,4.052221,6.710464,11.002974,17.822105,28.424428,44.427729
5,0.000000,0.000000,0.00000,0.000000,0.000000,0.500220,0.910602,1.657664,3.017618,5.493285,10.000000
6,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
S_0 = 100
K = 95
T = 5
r = 0.04
sig = 0.1
N = 10

delta_t = T/N
u = exp(sig*sqrt(delta_t))
d = 1/u
R = exp(r*delta_t)
q = (R - d) / (u - d)
BT = binomial_tree(S_0, N+1, u, d, delta_t)

In [13]:
BT

,0,1,2,3,4,5,6,7,8,9,10
0,100.0,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516,164.045681,176.065417,188.965846,202.811498
1,0.0,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516,164.045681,176.065417
2,0.0,0.000000,86.812345,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644,142.411902,152.846516
3,0.0,0.000000,0.000000,80.885789,86.812345,93.173142,100.000000,107.327066,115.190991,123.631111,132.689644
4,0.0,0.000000,0.000000,0.000000,75.363832,80.885789,86.812345,93.173142,100.000000,107.327066,115.190991
5,0.0,0.000000,0.000000,0.000000,0.000000,70.218850,75.363832,80.885789,86.812345,93.173142,100.000000
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,65.425109,70.218850,75.363832,80.885789,86.812345
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.958630,65.425109,70.218850,75.363832
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,56.797071,60.958630,65.425109
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,52.919616,56.797071


In [14]:
Euro_put = option_price(S_0, K, sig, N+1, q, BT, R, 'put', 'European')

C:\Users\김수빈\AppData\Local\Temp\ipykernel_32960\561204226.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Euro_tree[N-i][index] = 0.0
C:\Users\김수빈\AppData\Local\Temp\ipykernel_32960\561204226.py:9: FutureWarning: ChainedAssignmentError: 

In [15]:
Euro_put

,0,1,2,3,4,5,6,7,8,9,10
0,1.284795,0.689057,0.309074,0.104315,0.020176,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,2.347152,1.359637,0.667072,0.250201,0.054899,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,4.119863,2.587441,1.397964,0.589262,0.149377,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,6.896479,4.709780,2.821442,1.354326,0.406445,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,10.913415,8.111514,5.419231,3.007472,1.105913,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,16.172357,13.036755,9.731764,6.339529,3.009126,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.270944,19.248781,15.911165,12.233085,8.187655
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28.509001,25.849888,22.900024,19.636168
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.477926,32.160244,29.574891
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.199258,38.202929


In [16]:
Amer_put = option_price(S_0, K, sig, N+1, q, BT, R, 'put', 'American')

C:\Users\김수빈\AppData\Local\Temp\ipykernel_32960\561204226.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Euro_tree[N-i][index] = 0.0
C:\Users\김수빈\AppData\Local\Temp\ipykernel_32960\561204226.py:9: FutureWarning: ChainedAssignmentError: 

In [17]:
Amer_put

,0,1,2,3,4,5,6,7,8,9,10
0,2.180484,0.996401,0.389950,0.116707,0.020176,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,4.271905,2.061078,0.866474,0.283918,0.054899,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,8.187655,4.163606,1.884316,0.681005,0.149377,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,14.114211,8.187655,3.991835,1.603954,0.406445,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,19.636168,14.114211,8.187655,3.686696,1.105913,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,24.781150,19.636168,14.114211,8.187655,3.009126,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,29.574891,24.781150,19.636168,14.114211,8.187655
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,34.041370,29.574891,24.781150,19.636168
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,38.202929,34.041370,29.574891
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,42.080384,38.202929
